# Tools and Routing

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.tools import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [11]:
search.run("sf")

'42f'

In [12]:
from pydantic import BaseModel, Field

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    import requests
    from datetime import datetime, UTC

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
        "timezone": "UTC",  # helps make returned times consistent
    }

    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    results = response.json()

    current_utc_time = datetime.now(UTC)

    def _parse_to_utc(ts: str) -> datetime:
        # Handle "Z" suffix if present
        dt = datetime.fromisoformat(ts.replace("Z", "+00:00"))
        # If Open-Meteo returned a naive timestamp, assume it's UTC
        if dt.tzinfo is None:
            return dt.replace(tzinfo=UTC)
        # Otherwise normalize to UTC
        return dt.astimezone(UTC)

    time_list = [_parse_to_utc(t) for t in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]

    closest_time_index = min(
        range(len(time_list)),
        key=lambda i: abs(time_list[i] - current_utc_time),
    )

    current_temperature = temperature_list[closest_time_index]
    return f"The current temperature is {current_temperature}°C"

In [13]:
get_current_temperature.name

'get_current_temperature'

In [14]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [15]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_tool

In [17]:
convert_to_openai_tool(get_current_temperature)

{'type': 'function',
 'function': {'name': 'get_current_temperature',
  'description': 'Fetch current temperature for given coordinates.',
  'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
     'type': 'number'},
    'longitude': {'description': 'Longitude of the location to fetch weather data for',
     'type': 'number'}},
   'required': ['latitude', 'longitude'],
   'type': 'object'}}}

In [18]:
get_current_temperature.invoke({"latitude": 13, "longitude": 14})

'The current temperature is 33.9°C'

In [19]:

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    import wikipedia
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [20]:
search_wikipedia.name

'search_wikipedia'

In [21]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [22]:
convert_to_openai_tool(search_wikipedia)

{'type': 'function',
 'function': {'name': 'search_wikipedia',
  'description': 'Run Wikipedia search and get page summaries.',
  'parameters': {'properties': {'query': {'type': 'string'}},
   'required': ['query'],
   'type': 'object'}}}

In [23]:
search_wikipedia.invoke({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Vector database\nSummary: A vector database, vector store or vector search engine is a database that stores and retrieves embeddings of data in vector space. Vector databases typically implement approximate nearest neighbor algorithms so users can search for records semantically similar to a given input, unlike traditional databases which primarily look up records by exact match. Use-cases for vector databases include similarity search, semantic search, multi-modal search, recommendations engines, object detection, and retrieval-augmented generation (RAG).\nVector embeddings are mathematical representations of data in a high-dime

### LangChain OpenAPI Tool Calling Using An OpenAPI Schema Example

In [24]:
import json

from langchain_openai import ChatOpenAI

from langchain_community.agent_toolkits.json.toolkit import JsonToolkit
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain_community.utilities.requests import TextRequestsWrapper


In [25]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [29]:
# Parse the OpenAPI spec (as JSON here) and wrap it in a JsonSpec for the toolkit.
spec_dict = json.loads(text)
json_spec = JsonSpec(dict_=spec_dict, max_value_length=4000)

# Requests wrapper (no auth needed for the Petstore example).
requests_wrapper = TextRequestsWrapper(headers={})


In [30]:
# Build a modern set of Tools for interacting with an OpenAPI-described API.
#
# NOTE: OpenAPIToolkit.get_tools() includes a `json_explorer` tool implemented as an *agent*.
# That agent internally uses stop sequences, which GPT-5-mini rejects (no `stop` support).
# To stay fully modern (tools-first) *and* compatible with GPT-5-mini, we compose:
#   - RequestsToolkit: deterministic HTTP request tools (GET/POST/PATCH/PUT/DELETE)
#   - JsonToolkit: deterministic tools for exploring the OpenAPI JSON spec
# This avoids any legacy agent/function-calling shims and avoids passing `stop`.

llm = ChatOpenAI(model="gpt-5-mini", disabled_params={"stop": None})

requests_toolkit = RequestsToolkit(
    requests_wrapper=requests_wrapper,
    allow_dangerous_requests=True,
)
json_toolkit = JsonToolkit(spec=json_spec)

openapi_tools = [*requests_toolkit.get_tools(), *json_toolkit.get_tools()]
[len(openapi_tools), [t.name for t in openapi_tools]]


[7,
 ['requests_get',
  'requests_post',
  'requests_patch',
  'requests_put',
  'requests_delete',
  'json_spec_list_keys',
  'json_spec_get_value']]

In [31]:
# Bind the OpenAPI tools to the model (OpenAI "tools" schema).
llm_with_openapi_tools = llm.bind_tools(openapi_tools)


In [87]:
llm_with_openapi_tools.invoke("What are three pet names? (Call the API if you need to.)")


AIMessage(content='Here are three pet names: Bella, Max, Luna.\n\nWant names for a specific type (dog, cat, bird) or personality?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 706, 'total_tokens': 807, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ctj6p9BHTmSokKljRABR4U3etmftS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b8119-4009-7562-be6a-b43bf8f937fc-0', usage_metadata={'input_tokens': 706, 'output_tokens': 101, 'total_tokens': 807, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [32]:
llm_with_openapi_tools.invoke("Tell me about the pet with id 42. (Use the API.)")


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 706, 'total_tokens': 804, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CuJ1JXsZ4VLXn5OkbxJiv4lcTzC4I', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8953-9bc4-7ed3-b6c1-1791b26ac6db-0', tool_calls=[{'name': 'requests_get', 'args': {'url': 'https://petstore.swagger.io/v2/pet/42'}, 'id': 'call_5eJwrEjqjmxfL4Uu6hcwfwFM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 706, 'output_tokens': 98, 'total_tokens': 804, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [33]:
# If the model chose to call a tool, you'll see tool_calls on the AIMessage.
msg = llm_with_openapi_tools.invoke("Look up the pet with id 42 and summarize it in one sentence.")
msg.tool_calls


[{'name': 'requests_get',
  'args': {'url': 'https://petstore.swagger.io/v2/pet/42'},
  'id': 'call_xRAipkXG1pfNIsizBupcAl5f',
  'type': 'tool_call'}]

In [39]:
# Helper: execute tool calls locally (single-step execution).
from langchain_core.messages import AIMessage, ToolMessage

def execute_tool_calls(ai_message: AIMessage, tools):
    tool_map = {t.name: t for t in tools}
    tool_messages = []
    for call in ai_message.tool_calls or []:
        name = call["name"]
        args = call.get("args") or {}
        print(f'call name: {call["name"]} and call args: {call["args"]}')
        result = tool_map[name].invoke(args)
        tool_messages.append(ToolMessage(content=str(result), tool_call_id=call["id"]))
    return tool_messages

tool_messages = execute_tool_calls(msg, openapi_tools)
tool_messages[:1]


call name: requests_get and call args: {'url': 'https://petstore.swagger.io/v2/pet/42'}


[ToolMessage(content='{"code":1,"type":"error","message":"Pet not found"}', tool_call_id='call_xRAipkXG1pfNIsizBupcAl5f')]

### Routing (modern tool-calling)

In lesson 3, we showed an example of **function calling** deciding between two candidate functions.

In LangChain v1 (and OpenAI's current API), the modern pattern is **tool calling**:
- You bind tools with `llm.bind_tools([...])`
- The model returns `AIMessage.tool_calls`
- Your application executes the tool(s) and optionally feeds results back to the model


In [40]:
from langchain_core.utils.function_calling import convert_to_openai_tool

tools = [search_wikipedia, get_current_temperature]
openai_tools_schema = [convert_to_openai_tool(t) for t in tools]
openai_tools_schema


[{'type': 'function',
  'function': {'name': 'search_wikipedia',
   'description': 'Run Wikipedia search and get page summaries.',
   'parameters': {'properties': {'query': {'type': 'string'}},
    'required': ['query'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'get_current_temperature',
   'description': 'Fetch current temperature for given coordinates.',
   'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
      'type': 'number'},
     'longitude': {'description': 'Longitude of the location to fetch weather data for',
      'type': 'number'}},
    'required': ['latitude', 'longitude'],
    'type': 'object'}}}]

In [41]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None}).bind_tools(tools)


In [42]:
model.invoke("What is the weather in San Francisco right now?")


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 184, 'total_tokens': 283, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CuJMierIQXX1xpsnMxaKzFMTgLnHi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8967-dc92-7813-8e1b-fe8232022668-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': 'call_jDl0GpRawQzZBHw48KnSNWZy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 184, 'output_tokens': 99, 'total_tokens': 283, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [43]:
model.invoke("What is LangChain?")


AIMessage(content='Short answer\n- LangChain is an open‑source framework for building applications that use large language models (LLMs). It provides reusable building blocks (chains, agents, memories, retrievers, vectorstores, etc.) so you can compose LLM calls with data, logic, and external tools.\n\nWhat it does (overview)\n- Connects LLMs to data and to actions: lets models query documents (RAG), call APIs, run code, and maintain conversational state.\n- Makes production LLM apps easier to build, test, and scale by providing abstractions for prompting, orchestration, and integrations.\n\nCore concepts\n- LLM wrappers: unified interfaces to models (OpenAI, Hugging Face, local models).\n- Prompts & prompt templates: parameterized prompt construction and management.\n- Chains: sequences/compositions of calls (prompt → model → postprocess → next step).\n- Agents & tools: let an LLM decide to call external tools/APIs or take multiple steps to solve a task.\n- Memory: short- and long-ter

In [47]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful but sassy assistant."),
    ("user", "{input}"),
])

chain = prompt | model


In [48]:
result = chain.invoke({"input": "What is the weather in SF right now?"})
type(result), result.tool_calls


(langchain_core.messages.ai.AIMessage,
 [{'name': 'get_current_temperature',
   'args': {'latitude': 37.7749, 'longitude': -122.4194},
   'id': 'call_iaVmg0oM9WvfHm6MhXN2EbxE',
   'type': 'tool_call'}])

In [49]:
# Execute the tool call(s) produced by the model.
tool_messages = execute_tool_calls(result, tools)
tool_messages


call name: get_current_temperature and call args: {'latitude': 37.7749, 'longitude': -122.4194}


[ToolMessage(content='The current temperature is 12.1°C', tool_call_id='call_iaVmg0oM9WvfHm6MhXN2EbxE')]

In [54]:
# (Optional) Send tool results back to the model to get a final natural-language answer.
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a helpful but sassy assistant."),
    HumanMessage(content="What is the weather in SF right now?"),
    result,
    *tool_messages,
]

final_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None})
final_llm.invoke(messages).content


'Right now in San Francisco it’s 12.1°C (53.8°F). Pretty brisk — a light jacket is recommended. \n\nWant current conditions (clouds, rain, wind) or a forecast for the day? I can pull that too.'

In [55]:
# A simple "router": if there are tool calls, execute them; otherwise return the text.
def route_once(ai_message, tools):
    if getattr(ai_message, "tool_calls", None):
        tool_messages = execute_tool_calls(ai_message, tools)
        return {"ai_message": ai_message, "tool_messages": tool_messages}
    return {"ai_message": ai_message, "tool_messages": []}

routed = route_once(chain.invoke({"input": "What is LangChain?"}), tools)
routed["ai_message"].content


'Short answer (because you asked nicely): LangChain is an open-source framework / SDK for building applications that use large language models (LLMs). Think of it as the glue, batteries, and instruction manual for turning an LLM into a useful app — chatbots, retrieval-augmented generation (RAG), multi-step agents, pipelines, and more.\n\nQuick breakdown — what LangChain gives you\n- Core idea: compose LLM calls with other pieces (prompts, data, tools, memory, logic) into reusable "chains" and "agents".\n- Key components:\n  - LLM wrappers: unify access to different models (OpenAI, Anthropic, local LLMs, etc.).\n  - Prompts & prompt templates: manage and parametrize prompts cleanly.\n  - Chains: compose multiple steps (e.g., call LLM → process → call LLM again).\n  - Agents: let the model decide which tools to call (search, calculator, browser, APIs) and orchestrate multi-step workflows.\n  - Memory: keep conversation or application state between calls.\n  - Connectors / Loaders: import

In [56]:
routed = route_once(chain.invoke({"input": "What is the weather in San Francisco right now?"}), tools)
routed["ai_message"].tool_calls


call name: get_current_temperature and call args: {'latitude': 37.7749, 'longitude': -122.4194}


[{'name': 'get_current_temperature',
  'args': {'latitude': 37.7749, 'longitude': -122.4194},
  'id': 'call_TJgyFFYZYSKA7x35ZHKJGkvt',
  'type': 'tool_call'}]

In [57]:
# Run the tool(s) and get a final answer in one helper.
def answer_with_tools(user_input: str, tools):
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None}).bind_tools(tools)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful but sassy assistant."),
        ("user", "{input}"),
    ])
    first = (prompt | llm).invoke({"input": user_input})
    tool_messages = execute_tool_calls(first, tools)
    if not tool_messages:
        return first.content

    final_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None})
    messages = [
        SystemMessage(content="You are a helpful but sassy assistant."),
        HumanMessage(content=user_input),
        first,
        *tool_messages,
    ]
    return final_llm.invoke(messages).content

answer_with_tools("What is the weather in San Francisco right now?", tools)


call name: get_current_temperature and call args: {'latitude': 37.7749, 'longitude': -122.4194}


'It’s 12.1°C (53.8°F) in San Francisco right now — pleasantly cool. \n\nI only have the current temperature from my check. Want current conditions (cloudy/rain/wind), humidity, an hourly forecast, or radar?'

In [58]:
answer_with_tools("What is LangChain?", tools)


'Short answer: LangChain is an open‑source framework for building applications that use large language models (LLMs). Think of it as a batteries‑included toolkit that connects LLMs to data, tools, memory, and orchestration so you can build chatbots, retrieval‑augmented QA, agents that call APIs, and other LLM-powered apps faster.\n\nWhat it gives you (high level)\n- Connectors: easy integrations with LLM providers (OpenAI, Anthropic, Hugging Face, etc.), vector databases (Pinecone, Chroma, FAISS, Weaviate, Milvus), search, file stores and other tools.\n- Chains: composable building blocks that link prompt templates, LLM calls, retrievers, and post‑processing into workflows.\n- Prompts & templates: structured prompt handling, few‑shot examples, and prompt optimization helpers.\n- Retrieval & RAG: retrievers + vectorstores for retrieval‑augmented generation (search docs and feed results into the LLM).\n- Agents & tools: let an LLM plan and call external tools/APIs (search, calculators, c

In [59]:
answer_with_tools("hi!", tools)


"Well hello! What's up — what can I help you with today?"

In [60]:
answer_with_tools("Search Wikipedia for LangChain and give me one sentence.", tools)


call name: search_wikipedia and call args: {'query': 'LangChain'}


'According to Wikipedia, LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications.'

In [61]:
answer_with_tools("What's the temperature in London right now?", tools)


call name: get_current_temperature and call args: {'latitude': 51.5074, 'longitude': -0.1278}


'It’s 2.0°C in London right now — that’s about 35.6°F. Brr… you’ll want a coat. Want a forecast or rain check, too?'

In [62]:
# You can also inspect the raw tool calls:
llm_tools = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None}).bind_tools(tools)
raw = (prompt | llm_tools).invoke({"input": "What's the weather in SF right now?"})
raw.tool_calls


[{'name': 'get_current_temperature',
  'args': {'latitude': 37.7749, 'longitude': -122.4194},
  'id': 'call_6sSSbGrle7IGisIrLarpEXoz',
  'type': 'tool_call'}]

In [63]:
execute_tool_calls(raw, tools)


call name: get_current_temperature and call args: {'latitude': 37.7749, 'longitude': -122.4194}


[ToolMessage(content='The current temperature is 12.1°C', tool_call_id='call_6sSSbGrle7IGisIrLarpEXoz')]

In [64]:
# Final answer after tool execution:
user_input = "What's the weather in SF right now?"
tool_msgs = execute_tool_calls(raw, tools)
final_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None})
final_llm.invoke([
    SystemMessage(content="You are a helpful but sassy assistant."),
    HumanMessage(content=user_input),
    raw,
    *tool_msgs,
]).content


call name: get_current_temperature and call args: {'latitude': 37.7749, 'longitude': -122.4194}


"It's 12.1°C (53.8°F) in San Francisco right now. I only have the temperature at the moment — want me to pull current conditions (clouds, wind, rain) or the forecast so you can decide whether to grab a jacket?"